# Assignment 2
Bradley Thompson - CS 510 Large Language Models PDX Winter 2024

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
MODELS = (
    "bigscience/bloom-560m",
    "bigscience/bloom-1b1",
    "bigscience/bloom-1b7",
    "bigscience/bloomz-560m",
    "bigscience/bloomz-1b1",
    "bigscience/bloomz-1b7",
)
ID_TO_LABEL = {
    0: "negative",
    1: "neutral",
    2: "positive",
}
LABEL_TO_ID = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

DEFAULT_PROMPT = "Asign any label of these options ['positive', 'negative', 'neutral'] to the following text: '%s'. Label: "

model_name = MODELS[0]
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [2]:
from datasets import load_dataset
dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english")
sample = 2
negative_texts = [ sample for sample in dataset["train"] if sample["label"] == LABEL_TO_ID["negative"] ]
#text = dataset["train"][sample]["text"]
text = negative_texts[2]
text

{'text': 'it looks like a beautiful night to throw myself off the Brooklyn Bridge ---@Tim_Hecht ',
 'label': 0}

In [9]:
from transformers import GenerationConfig

NEW_TOKENS = 5

config = {
    "min_new_tokens": 1,
    "max_new_tokens": NEW_TOKENS,
    "use_cache": False,
    # "do_sample": True,
    # "top_k": 2,
}

def output_with_classification(prompt: str) -> str:
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    gen_config: GenerationConfig = GenerationConfig.from_dict(config)
    output = model.generate(inputs, gen_config)[0]
    return tokenizer.decode(output[-NEW_TOKENS:])

for sample in negative_texts[:2]:
    input = sample["text"]
    print("Input: ", input)
    print("Actual: ", ID_TO_LABEL[sample["label"]])
    print("Classification: ", output_with_classification(DEFAULT_PROMPT % input))

Input:  okay i\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\u2019mon America aren\u2019t you sick of her yet? (sorry) 
Actual:  negative
Classification:   'Positive', '
Input:  The tragedy of only thinking up hilarious tweets for the Summer Olympics now is that in four years there may be no place for them. 
Actual:  negative
Classification:   'Positive', Label
